In [66]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd
import re
import sklearn
import numpy as np
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

# User Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)

### For each query we generate three results, two of them are based on the user's preference. The last one are on food categories that's less frequent

In [67]:
customer =  pd.read_csv('./customer.csv')
customer_rare =  pd.read_csv('./customer_rare.csv')

In [68]:
customer_array = np.load("customer_array.npy",allow_pickle=True)
customer_rare_array = np.load("customer_rare_array.npy",allow_pickle=True)
by_customer_rare = pd.read_csv('./by_customer_rare.csv')
by_customer = pd.read_csv('./by_customer.csv')

## fit KNN model

In [69]:
'''
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]
neigh = NearestNeighbors(2, 0.4)
neigh.fit(samples) 
neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=False)
'''
neigh = NearestNeighbors(2, 0.4)
neigh.fit(customer_array) 

neigh_rare = NearestNeighbors(2, 0.4)
neigh_rare.fit(customer_rare_array) 

C:\Users\anban\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=2, radius=0.4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\anban\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=2, radius=0.4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


NearestNeighbors(n_neighbors=2, radius=0.4)

## Predict

In [70]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.00347569, 9900.00366229]]), array([[29447,  5277]], dtype=int64))

In [71]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7888888888888889,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.0036272 , 9900.00381381]]), array([[29447,  5277]], dtype=int64))

In [72]:
neigh.kneighbors([[0, 0, 2, 0.0329, -78.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]], 2, return_distance=True)

(array([[9900.00384829, 9900.00423388]]), array([[29447, 29083]], dtype=int64))

## Give Recommendation

In [73]:
customer

,Unnamed: 0,Unnamed: 0.1,customer_id,gender,location_number,location_type,latitude_x,longitude_x,city_id,tag_1,...,tag_84,tag_85,tag_86,tag_87,tag_88,tag_89,tag_90,tag_91,grand_total,item_count
0,0,0,TCHWPBT,0,0,1,-96.44000,-67.20000,1.0,1,...,0,0,0,0,0,0,0,1,6.650000,2.289742
1,2,200,TCHWPBT,0,2,2,-0.12870,-78.56000,1.0,1,...,0,0,0,0,0,0,0,1,6.650000,2.289742
2,3,300,ZGFSYCZ,0,0,0,-0.17550,-78.56000,1.0,42,...,0,0,0,0,0,0,7,42,17.070732,2.656035
3,4,400,ZGFSYCZ,0,1,0,0.19120,-78.60000,1.0,42,...,0,0,0,0,0,0,7,42,17.070732,2.656035
4,6,600,ZGFSYCZ,0,3,0,0.04245,-78.60000,1.0,42,...,0,0,0,0,0,0,7,42,17.070732,2.656035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42548,49180,5800500,NCTU1O6,3,0,2,-0.70950,0.12463,1.0,0,...,0,0,0,0,0,0,0,0,8.400000,1.000000
42549,49181,5800700,MSEGQHZ,3,0,0,13.35000,-1.38700,1.0,0,...,0,0,0,0,0,0,0,0,6.000000,3.000000
42550,49182,5800900,ERHULM8,3,0,0,-1.14900,-0.17960,1.0,1,...,0,0,0,0,0,0,0,1,0.900000,3.000000
42551,49183,5801700,LE63M0S,1,0,2,-1.32600,0.05917,1.0,0,...,0,0,0,0,0,0,0,0,11.400000,2.000000


In [74]:
by_customer


,Unnamed: 0,customer_id,LOCATION_NUMBER,VENDOR
0,0,92PEE24,0,105 105 105
1,1,QS68UD8,0,294
2,2,MB7VY5F,0,83 83 83 83
3,3,KDJ951Y,0,90
4,4,BAL0RVT,0,83 83 83
...,...,...,...,...
43636,43636,3S6VG6R,1,199
43637,43637,ND4PIJL,0,907
43638,43638,1NRK5HF,0,105
43639,43639,QDXLWM7,1,28


In [75]:
by_customer = by_customer.set_index(['customer_id','LOCATION_NUMBER'])

In [76]:
by_customer.loc['A1U8ZU0', 1]

Unnamed: 0    24104
VENDOR           43
Name: (A1U8ZU0, 1), dtype: object

In [77]:
by_customer_rare = by_customer_rare.set_index(['customer_id','LOCATION_NUMBER'])

In [121]:
q = [[0, 0, 2, 0.0329, -3.6, 10000.0, 9.0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 6, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0,
       0.2222222222222222]]

def query(user_vecs, neigh, table, pool, k_value=3, return_num=1):
    dist, knn = neigh.kneighbors(user_vecs, k_value, return_distance=True)
    recommendation = []
    recommendation_weight = []
    #print(dist,knn)
    for k,i in enumerate(list(knn)):
        
        #print(pool.iloc[i])
        user_id = table.iloc[i]['customer_id'].values
        loc = table.iloc[i]['location_number'].values
        recommended_restaurant = []
        for id in zip(user_id,loc):
            #print(table.iloc[i]['customer_id'].values,table.iloc[i]['latitude_x'].values,table.iloc[i]['longitude_x'].values)
            s = pool.loc[id]['VENDOR']
            #print("str",str(s))
            recommended_restaurant.append(s.split(' '))
        confidence = {}
        for j in range(len(recommended_restaurant)):
            for restaurant in recommended_restaurant[j]:
                if restaurant not in confidence:
                    confidence[restaurant]=0.
                confidence[restaurant]+=1./dist[0][j]
        p = np.asarray(list(confidence.values()))
        p/=np.sum(p)
        keys = list(confidence.keys())
        recommendation.append(np.random.choice(len(confidence.keys()), return_num, p=p,replace=True))
        ret = [keys[recommendation[-1][n]]for n in range(return_num)]
        #print("generate recommendation for %d'th input..."%k)
        #print("--Recommendation is restaurant: "+str(ret))
        return ret
        

def generate_recommendation(user_vecs):
    l1 = query(user_vecs, neigh, customer, by_customer, k_value=5, return_num=3)
    l2 = query(user_vecs, neigh_rare, customer_rare, by_customer_rare, k_value=5, return_num=1)
    if l2[0] in l1:
        return l1
    else:
        return [l1[0],l1[1],l2[0]]
generate_recommendation(q)

['58SUVOG' 'YWCKP7T' 'UYJ25V9' 'HN27QZ5' 'FMO99SN'] [-0.5405  0.6147 -1.255   0.1761 -1.146 ] [ 0.1771  0.542  -0.1261  0.6045 -0.0743]
['58SUVOG' 'YWCKP7T' 'UYJ25V9' 'HN27QZ5' 'FMO99SN'] [-0.5405  0.6147 -1.255   0.1761 -1.146 ] [ 0.1771  0.542  -0.1261  0.6045 -0.0743]
['58SUVOG' 'YWCKP7T' 'UYJ25V9' 'HN27QZ5' 'FMO99SN'] [-0.5405  0.6147 -1.255   0.1761 -1.146 ] [ 0.1771  0.542  -0.1261  0.6045 -0.0743]
['58SUVOG' 'YWCKP7T' 'UYJ25V9' 'HN27QZ5' 'FMO99SN'] [-0.5405  0.6147 -1.255   0.1761 -1.146 ] [ 0.1771  0.542  -0.1261  0.6045 -0.0743]
['58SUVOG' 'YWCKP7T' 'UYJ25V9' 'HN27QZ5' 'FMO99SN'] [-0.5405  0.6147 -1.255   0.1761 -1.146 ] [ 0.1771  0.542  -0.1261  0.6045 -0.0743]
['YWCKP7T' 'HN27QZ5' 'B0JTR7Q' 'SLXOGGQ' 'C6C18GA'] [ 0.6147  0.1761  0.578  -0.1067  0.5024] [ 0.542   0.6045  0.526   0.1219 -0.1274]
['YWCKP7T' 'HN27QZ5' 'B0JTR7Q' 'SLXOGGQ' 'C6C18GA'] [ 0.6147  0.1761  0.578  -0.1067  0.5024] [ 0.542   0.6045  0.526   0.1219 -0.1274]
['YWCKP7T' 'HN27QZ5' 'B0JTR7Q' 'SLXOGGQ' 'C6C18G

['259', '160', '160']

# Generate Recommendation for Group 

In [116]:
def generate_recommendation_for_group(user_vecs):
    hybrid_preference = np.sum(np.asarray(user_vecs)[:,6:-2],0).tolist()
    vote = []
    vote_rare = []
    count_freq = {}
    for user_vec in user_vecs:
        user_vec[6:-2]=hybrid_preference
        recommendation = generate_recommendation([user_vec])
        vote.extend(recommendation[:2])
        vote_rare.extend(recommendation[2:])
    for k in vote:
        if k not in count_freq:
            count_freq[k]=0
        count_freq[k]+=1
    count_freq = sorted(count_freq.items(),key=lambda x:x[1], reverse=True)
    ret =[i[0] for i in count_freq[:2]]
    count_freq = {}
    for k in vote_rare:
        if k not in count_freq:
            count_freq[k]=0
        count_freq[k]+=1
    count_freq = sorted(count_freq.items(),key=lambda x:x[1], reverse=True)
    ret.append(count_freq[0][0])
    #print("generate recommendation for group...")
    #print("--Recommendation is restaurant: "+str(ret))
    return ret

In [117]:
generate_recommendation_for_group(customer_array[:10].tolist())

['216', '295', '300']

# Item Base Recommendation

## Assign weights to each feature

### city_id is supper critical, so we assign large weight. (customer_array[:,5]*=100.) Note that latitude_x and	longitude_x is also important but originally has large variance, so we keep them the same.)
### We want to amplify the customer's preference (customer_array[:,6]*=3.)
### grand_total and	item_count may not be so important, we dilute them (customer_array[:,-2:]/=3.)